In [1]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
from Load import json_to_df
import numpy as np

In [2]:
import re
def cleanText(text):
    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = replace_contraction(text)
    text = replace_links(text, "link")
    text = remove_numbers(text)
    text = re.sub(r'[,!@#$%^&*)(|/><";:.?\'\\}{]',"",text)
    text = text.lower()
    return text

In [3]:
def replace_contraction(text):
    contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'can not'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    return text
def replace_links(text, filler=' '):
        text = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*',
                      filler, text).strip()
        return text
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [4]:
df_music = json_to_df("data/music_reviews_train.json")
df_music_dev = json_to_df("data/music_reviews_dev.json")

df_music=df_music.replace({"sentiment":{"positive":1,"negative":0}})
df_music_dev=df_music_dev.replace({"sentiment":{"positive":1,"negative":0}})

df_music = df_music[["summary","sentiment"]].dropna()
df_music_dev = df_music_dev[["summary","sentiment"]].dropna()
df_music["summary"] = df_music["summary"].apply(cleanText)
df_music_dev["summary"] = df_music_dev["summary"].apply(cleanText)


X = np.array(df_music["summary"])
y = np.array(df_music["sentiment"])

X_dev = np.array(df_music_dev["summary"])
y_dev = np.array(df_music_dev["sentiment"])

In [5]:
embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [6]:
import tensorflow.compat.v1.keras as keras 
from tensorflow.compat.v1.keras.layers import Input, Lambda, Dense
from tensorflow.compat.v1.keras.models import Model
import tensorflow.compat.v1.keras.backend as K

In [7]:
GPU_OPTIONS = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
CONFIG = tf.ConfigProto(gpu_options=GPU_OPTIONS)
sess = tf.Session(config = CONFIG)

In [8]:
def build_model(): 
    input_text = Input(shape=(1,), dtype="string")
    embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
    dense = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001))(embedding)
    pred = Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='binary_crossentropy', 
                  optimizer=keras.optimizers.SGD(momentum=0.1, nesterov=True), 
                  metrics=['accuracy'])
    return model

In [9]:
model_elmo = build_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
model_elmo.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [11]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model_elmo.fit(X, y, epochs=5, batch_size=256, validation_data=(X_dev,y_dev))
    model_elmo.save_weights('./model_elmo_weights.h5')

Train on 99969 samples, validate on 9999 samples
Epoch 1/5
12800/99969 [==>...........................] - ETA: 9:18 - loss: 1.0166 - accuracy: 0.6559

KeyboardInterrupt: 

In [ ]:
X